In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

# n.Data were preprocess from this kernel prhttps://www.kaggle.com/holydark30/cleaned-data
# If decide to use this one,duplicate code below doesn't need to run.
#cleaned_train = pd.read_csv('/kaggle/input/tj-data/cleaned_train.csv')
#cleaned_test = pd.read_csv('/kaggle/input/tj-data/cleaned_test.csv')

In [ ]:
train = pd.read_csv('/kaggle/input/tj19data/train.csv')
test = pd.read_csv('/kaggle/input/tj19data/test.csv')
dem = pd.read_csv('/kaggle/input/tj19data/demo.csv')
txn = pd.read_csv('/kaggle/input/tj19data/txn.csv')

In [ ]:
txn[['n3','n4']].describe()

In [ ]:
txn.describe()


In [ ]:
txn[txn['n4'] < 0]

In [ ]:
txn.dtypes
# Convert type to make data process faster.
txn.astype({'id': 'int32', 'old_cc_label':'int8', 
            'c5': 'uint8','c6':'int16','c7':'uint8',
            'n3':'uint16','n5':'uint16', 'n6':'uint16','n7':'uint16'}).dtypes

In [ ]:
txn['month'] = pd.to_datetime(txn['n3'], format='%j').dt.strftime('%m')

In [ ]:
limit = dem['n2'].dropna().unique() 
dem['n1'].fillna(0, inplace=True)
def find_nearest_above(target, my_array=limit):
    diff = my_array - target
    mask = np.ma.less_equal(diff, 0)
    # We need to mask the negative differences and zero
    # since we are looking for values above
    if np.all(mask):
        return None # returns None if target is greater than any value
    masked_diff = np.ma.masked_array(diff, mask)
    return my_array[masked_diff.argmin()]
dem['n2'] = dem.apply(lambda x: find_nearest_above(target=x['n1']) if pd.isnull(x['n2']) else x['n2'], axis=1)
dem['rat_n1_n2'] = dem['n1']/dem['n2']

In [ ]:
def cleaning(df):
    temp = df.copy()
    temp = pd.merge(temp, txn.groupby(['id'])['old_cc_label'].agg(lambda x: pd.Series.mode(x)[0]), on="id",how="left")
    temp = pd.merge(temp, txn.groupby(['id'])['n7'].agg(lambda x: pd.Series.mode(x)[0]), on="id",how="left")
    temp = pd.merge(temp, txn.groupby(['id'])['c5'].agg(lambda x: pd.Series.mode(x)[0]), on="id",how="left")
    temp = pd.merge(temp, txn.groupby(['id'])['c6'].agg(lambda x: pd.Series.mode(x)[0]), on="id",how="left")
    temp = pd.merge(temp, txn.groupby(['id'])['c7'].agg(lambda x: pd.Series.mode(x)[0]), on="id",how="left")
    
    group_month = txn.groupby(['id','month'])[['n4']].agg(['mean' , 'sum'])
    group_month.columns = ['n4_month_mean','n4_month_sum']
    group_month.reset_index(inplace=True)
    
    uniq_month = group_month['month'].unique()
    for month in uniq_month:
        t = group_month[group_month['month'] == month].copy()
        t.drop('month',axis=1, inplace=True)
        t.columns = ['id','n4_month_mean_'+month,'n4_month_sum_'+month]
        temp = pd.merge(temp,t, on="id",how="left")
        
    group_month = txn.groupby(['id'])[['n4']].agg(['mean' , 'sum']).reset_index()
    group_month.columns = ['id','n4_count_mean','n4_month_sum']
    temp = pd.merge(temp, group_month,on="id",how="left")
    
    # Transaction frequency 
    count = txn.groupby('id').size().reset_index()
    count.columns = ['id','freq']
    temp = pd.merge(temp, count, on="id",how="left")
    
    # Work on feature engineer n5, and n6.

    t = txn.groupby(['id'])[['n5','n6']].agg(['sum','mean']).reset_index()
    t.columns = ['id', 'n5_sum','n5_mean','n6_sum','n6_mean']
    
    temp = pd.merge(temp, t, on="id", how="left")
    
    # Transaction used on certain service/purchase.
    #txt = txn['t0'].value_counts()
    #len(txn)
    # Select top 20 common consumption,
    #txt_feat = list(txt[:20].index)

    
    temp = pd.merge(temp, dem, on="id", how="left")
    temp.fillna(0, inplace=True)
    print(len(temp))
    return temp


In [ ]:
cleaned_train = cleaning(train)
cleaned_test = cleaning(test)

In [ ]:
(dem['n1'].fillna(0)/dem['n2'].dropna()).describe() 
# The ratio between n1 and n2 has a mean of lesser than 1, however max as large as 725...
# if n1 NaN is 0, and n2 can be dropped as low


In [ ]:
dem[dem['n1'] > dem['n2']]
# There are some case n1 > n2.. Possibly outlier?

In [ ]:
labels_w = np.array(train['label'].value_counts().sort_index()/len(train))
labels_w

In [ ]:
labels = np.sort(train['label'].unique())
labels

In [ ]:
cleaned_train = cleaned_train.append([cleaned_train[cleaned_train['label'] == 5]]*4,ignore_index=True)

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import log_loss
train = pd.read_csv('/kaggle/input/tj19data/train.csv')
w = train['label'].value_counts().sort_index() / len(train)
class_weights = list()
xgb_cw = dict()
for i in range(13):
    xgb_cw[i] = w[i]
    w_d = {0:1, 1:w[i]}
    class_weights.append(w_d)
    

def wmcl(F_prob, A):
    samp_w = compute_sample_weight(xgb_cw, A)
    return np.mean(log_loss(A, F_prob,sample_weight=samp_w))


def xgb_wmcl(y_pred, y_train):
    label = y_train.get_label()
    return 'logloss', wmcl(y_pred, label)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(cleaned_train.drop(['label', 'id'], axis=1), 
                                                    cleaned_train['label'], 
                                                    test_size=0.2, random_state=42, 
                                                    stratify=cleaned_train['label'])

#xgb_model = xgb.XGBClassifier(random_state=42)

#xgb_model.fit(X_train, y_train, sample_weight=compute_sample_weight(xgb_cw, y_train))



In [ ]:
X_test

In [ ]:
#preds_prob = xgb_model.predict_proba(X_train)
#print("Train metric " + str(wmcl(preds_prob,y_train.values)))


#preds_prob = xgb_model.predict_proba(X_test)
#print("Test metric " + str(wmcl(preds_prob, y_test.values)))

In [ ]:
cat_feat = ['c0','c1','c2','c3','c4','c5','c6','c7']
xgb_train = xgb.DMatrix(X_train.values, label=y_train.values, weight=compute_sample_weight(xgb_cw, y_train.values))
xgb_val = xgb.DMatrix(X_test.values)
xgb_eval = xgb.DMatrix(X_test.values, label=y_test.values, weight=compute_sample_weight(xgb_cw, y_test.values))


"""
N_FOLDS = 5

del hyperparameters['n_estimators']
hyperparameters['seed']= 42
hyperparameters['silent']= True
hyperparameters['nthread']= -1
hyperparameters['num_class'] = 13

# Perform cross validation with early stopping
cv_results = xgb.cv(hyperparameters, 
                    xgb_train, num_boost_round = 100, 
                    nfold = N_FOLDS, stratified=True,feval=xgb_wmcl,
                    early_stopping_rounds = 100,
                    verbose_eval = False, seed = 42, maximize=False)
max_idx = np.argmin(cv_results['test-logloss-mean'])
# Extract the best score

# Boosting rounds that returned the highest cv score
n_estimators = max_idx+1

# Add the number of estimators to the hyperparameters
hyperparameters['n_estimators'] = n_estimators

# Highest score
best = cv_results['test-logloss-mean'][max_idx]

# Standard deviation of best score
best_std = cv_results['test-logloss-std'][max_idx]


print('The most optimized logloss in cross validation was {:.5f} with std of {:.5f}.'.format(best, best_std))
print('The ideal number of iterations was {}.'.format(n_estimators)) """



In [ ]:
csw = compute_sample_weight(xgb_cw, y_train.values)
e_csw = compute_sample_weight(xgb_cw, y_test.values)

In [ ]:
from hyperopt import hp
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer

N_FOLDS = 5

def objective(hyperparameters):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization.
       Writes a new line to `outfile` on every iteration"""
    
    hyperparameters['silent'] = True
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Using early stopping to find number of trees trained

    hyperparameters['n_estimators']=300
    
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['max_depth', 'min_child_weight','random_state']:
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])

    start = timer()
        
    hyperparameters['seed']= 42
    hyperparameters['silent']= True
    hyperparameters['nthread']= -1
    
    
    # Perform cross validation with early stopping
    model = xgb.XGBClassifier(**hyperparameters)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], 
              early_stopping_rounds=25, sample_weight=csw,
              eval_metric=xgb_wmcl,
              sample_weight_eval_set=[e_csw])

    # Highest score
    loss = wmcl(model.predict_proba(X_test), y_test)


    # Boosting rounds that returned the highest cv score
    n_estimators = model.best_iteration

    # Add the number of estimators to the hyperparameters
    hyperparameters['n_estimators'] = n_estimators


    run_time = timer() - start

    # Loss must be minimized
    best_score = 100-loss
    

    # Write to the csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time, best_score])
    of_connection.close()

    # Dictionary with information for evaluation
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}



In [ ]:
# Define the search space
space = {
    'num_class':13,
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.3, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
    'max_depth': hp.quniform('max_depth', 5, 15, 1),
    'random_state': 42,
    'objective':'multi:softprob'
}


In [ ]:
from hyperopt import tpe
from hyperopt import Trials

# Create the algorithm
tpe_algorithm = tpe.suggest
# Record results
trials = Trials()

# Create a file and open a connection
OUT_FILE = 'bayes_test.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 2

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()

In [ ]:

"""
from hyperopt import fmin
# Global variable
global  ITERATION

ITERATION = 2
MAX_EVALS = 30
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

best

trials_dict = sorted(trials.results, key = lambda x: x['loss'])
results = pd.read_csv(OUT_FILE)

"""

In [ ]:
import ast
import shap
def evaluate(results, name):
    """Evaluate model on test data using hyperparameters in results
       Return dataframe of hyperparameters"""
    
    new_results = results.copy()
    # String to dictionary
    new_results['hyperparameters'] = new_results['hyperparameters'].map(ast.literal_eval)
    
    # Sort with best values on top
    new_results = new_results.sort_values('score', ascending = False).reset_index(drop = True)
    
    # Print out cross validation high score
    print('The highest cross validation score from {} was {:.5f} found on iteration {}.'.format(name, new_results.loc[0, 'score'], new_results.loc[0, 'iteration']))
    
    # Use best hyperparameters to create a model
    hyperparameters = new_results.loc[0, 'hyperparameters']
    model = xgb.XGBClassifier(**hyperparameters)
    model.fit(X_train, y_train, 
              early_stopping_rounds=25, 
              sample_weight=csw)
    
    # Train and make predictions
    preds = model.predict(X_test)
    
    print('log-loss from {} on test data = {:.5f}.'.format(name, wmcl(model.predict_proba(X_test), y_test)))
    
    # Create dataframe of hyperparameters
    hyp_df = pd.DataFrame(columns = list(new_results.loc[0, 'hyperparameters'].keys()))

    # Iterate through each set of hyperparameters that were evaluated
    for i, hyp in enumerate(new_results['hyperparameters']):
        hyp_df = hyp_df.append(pd.DataFrame(hyp, index = [0]), 
                               ignore_index = True)
        
    # Put the iteration and score in the hyperparameter dataframe
    hyp_df['iteration'] = new_results['iteration']
    hyp_df['score'] = new_results['score']
    
    return hyp_df

#bayes_results = evaluate(results, name = 'Bayesian')
#bayes_results

In [ ]:
new_results = results.copy()
# String to dictionary
new_results['hyperparameters'] = new_results['hyperparameters'].map(ast.literal_eval)

# Sort with best values on top
new_results = new_results.sort_values('score', ascending = False).reset_index(drop = True)

# Use best hyperparameters to create a model
hyperparameters = new_results.loc[0, 'hyperparameters']




In [ ]:
hyperparameters = {'colsample_bytree': 0.3193923647239065,
 'gamma': 0.33134122637800645,
 'learning_rate': 0.0824265818290931,
 'max_depth': 13,
 'min_child_weight': 3,
 'num_class': 13,
 'objective': 'multi:softprob',
 'random_state': 42,
 'reg_alpha': 0.4137163898530162,
 'reg_lambda': 0.6494331857571901,
 'silent': True,
 'n_estimators': 507,
 'seed': 42,
 'nthread': -1}

In [ ]:
model = xgb.XGBClassifier(**hyperparameters)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], 
          early_stopping_rounds=25, sample_weight=csw,
          eval_metric=xgb_wmcl,
          sample_weight_eval_set=[e_csw])

# Highest score
loss = wmcl(model.predict_proba(X_test), y_test)


In [ ]:
import shap
# Use Tree SHAP algorithms to explain the output of ensemble tree models
explainer = shap.TreeExplainer(model)


In [ ]:
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, feature_names=X_test.columns)

In [ ]:
# The first argument is the index of the feature we want to plot
# The second argument is the matrix of SHAP values (it is the same shape as the data matrix)
# The third argument is the data matrix (a pandas dataframe or numpy array)
shap.dependence_plot(0, shap_values, X_test)

In [ ]:
preds = model.predict_proba(cleaned_test.drop('id',axis=1))
col = ['class'+str(i) for i in range(13)]
sample_sub = pd.DataFrame(preds, columns=col)
sample_sub.insert(0,'id',cleaned_test['id'])
sample_sub.to_csv('O_1033.csv', index = False, header=True)

In [ ]:
sns.countplot(train['label'], order=[10,12,0,11,8,4,7,9,2,6,3,5],orient='')